In [1]:
%matplotlib inline
%autosave 0
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import matplotlib.pyplot as plt
import cv2
import simpleaudio as sa

model = ResNet50(weights='imagenet')
%autosave 60

Autosave disabled


Using TensorFlow backend.


Autosaving every 60 seconds


In [5]:
def crop_center(img,cropx,cropy):
    """Crop to the center (cropx, cropy) pixels of a multi-channel image."""
    y,x,_ = img.shape
    startx = x//2-(cropx//2)
    starty = y//2-(cropy//2)    
    return img[starty:starty+cropy,startx:startx+cropx,:]

In [34]:
def meow():
    """Play a cat meow sound."""
    wave_obj = sa.WaveObject.from_wave_file("meow.wav")
    play_obj = wave_obj.play()
    play_obj.wait_done()

# Test
meow()

In [38]:
# setup capture
camera = cv2.VideoCapture(0)

# reduce frame size to speed it up
w = 256
h = 256
c = 3
camera.set(cv2.CAP_PROP_FRAME_WIDTH, w) 
camera.set(cv2.CAP_PROP_FRAME_HEIGHT, h)

CAT_LABEL = 'tabby'

# capture loop
while True:
    # get frame
    ret, frame = camera.read()
        
    # mirror the frame (my camera mirrors by default)
    frame = cv2.flip(frame, 1)
    
    # Get a 224x224 np array by cropping into the center of the frame
    x = np.asarray(frame[:,:], dtype='d')
    x = crop_center(x, 224, 224)
    
    # Preprocess the array
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    # Make predictions
    preds = model.predict(x)
    
    # decode the results into a list of tuples (class, description, probability)
    # (one such list for each sample in the batch)
    top_three = decode_predictions(preds, top=3)[0]
    # print('Predicted:', top_three)
    
    # meow if cat detected
    if CAT_LABEL in [a[1] for a in top_three]:
        meow()
        
    # Show frame
    cv2.imshow('frame', frame)
    
    # Get key input
    k = cv2.waitKeyEx(250)
    
    # exit on ESC press
    if k == 27:
        break
    # p to print a video still in the notebook    
    elif k == 112:
        imshow(frame)
    elif k >= 0:
        print(k)
    
# clean up
cv2.destroyAllWindows()
camera.release()
cv2.waitKey(1) # extra waitKey sometimes needed to close camera window

-1